In [3]:
!pip install imutils
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
#credential : 4/AAC2-0LfNtfgM8ebYRv9fkZjbZAZVnlbl8WF801p8kfe_Rs5EYjqo5Q
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [4]:
file_list = drive.ListFile({'q': "'1tIbNp1wx47x9fn9ixLLg7_lbIaSOhmOg' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: training.ipynb, id: 1eeCEv262hvCBb18jKbuFPaNMNSIhpp6f
title: training_valid_padding.ipynb, id: 133onR0c2SUBcXJjL2wmNVy4TyNFOIqMP
title: val_data_train.npy, id: 1g1BZpBDPrurHd5FlSijF6BSmOPcNGe9q
title: val_data_label.npy, id: 1nZ6miRkOPJVG5bqn1XgCfWOQNxOaWrjK
title: train_mask.npy, id: 12IhPKN2ccsckFuIjauZYPZUbkb10E0k8
title: train_img.npy, id: 1ONPUkblwOD4BEYAZnsp5UuPq9DfNO2av


In [0]:
train_downloaded = drive.CreateFile({'id': '1g1BZpBDPrurHd5FlSijF6BSmOPcNGe9q'})
train_downloaded.GetContentFile('val_data_train.npy')
train_downloaded = drive.CreateFile({'id': '1nZ6miRkOPJVG5bqn1XgCfWOQNxOaWrjK'})
train_downloaded.GetContentFile('val_data_label.npy')
train_downloaded = drive.CreateFile({'id': '12IhPKN2ccsckFuIjauZYPZUbkb10E0k8'})
train_downloaded.GetContentFile('train_mask.npy')
train_downloaded = drive.CreateFile({'id': '1ONPUkblwOD4BEYAZnsp5UuPq9DfNO2av'})
train_downloaded.GetContentFile('train_img.npy')


In [1]:
ls


datalab/               train_img.npy   val_data_label.npy
model-dsbowl2018-1.h5  train_mask.npy  val_data_train.npy


In [0]:
# from preprocessing import read_train_data,read_test_data,prob_to_rles,mask_to_rle,resize,np
import numpy as np
import pandas as pd
import os
import sys
import cv2
from skimage.io import imread, imshow, imread_collection, concatenate_images
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint

def train_model():
  X_train=np.load('./train_img.npy')
  Y_train=np.load('./train_mask.npy')
  val_data=np.load('./val_data_train.npy')
  val_label=np.load('./val_data_label.npy')

  Y_train=Y_train.reshape(len(Y_train),Y_train[0].shape[0],Y_train[0].shape[1],1)
  val_label=val_label.reshape(len(val_label),val_label[0].shape[0],val_label[0].shape[1],1)

  print (X_train[0].shape, "training data shape")  
  print (Y_train[0].shape, "training label shape")

  smooth = 1.

  # Metric function
  def dice_coef(y_true, y_pred):
      y_true_f = K.flatten(y_true)
      y_pred_f = K.flatten(y_pred)
      intersection = K.sum(y_true_f * y_pred_f)
      return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

  # Loss funtion
  def dice_coef_loss(y_true, y_pred):
      return -dice_coef(y_true, y_pred)

  inputs = Input((None,None, 3))
  
  s = Lambda(lambda x: x / 255) (inputs)
  c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
  c1 = Dropout(0.1) (c1)
  c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
  p1 = MaxPooling2D((2, 2)) (c1)
  c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
  c2 = Dropout(0.1) (c2)
  c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
  p2 = MaxPooling2D((2, 2)) (c2)

  c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
  c3 = Dropout(0.2) (c3)
  c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
  p3 = MaxPooling2D((2, 2)) (c3)

  c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
  c4 = Dropout(0.2) (c4)
  c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
  p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

  c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
  c5 = Dropout(0.3) (c5)
  c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

  u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
  u6 = concatenate([u6, c4])
  c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
  c6 = Dropout(0.2) (c6)
  c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

  u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
  u7 = concatenate([u7, c3])
  c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
  c7 = Dropout(0.2) (c7)
  c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

  u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
  u8 = concatenate([u8, c2])
  c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
  c8 = Dropout(0.1) (c8)
  c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

  u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
  u9 = concatenate([u9, c1], axis=3)
  c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
  c9 = Dropout(0.1) (c9)
  c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

  outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

  model = Model(inputs=[inputs], outputs=[outputs])
  model.compile(optimizer='adam',loss='binary_crossentropy', metrics=[dice_coef])
  model.summary()



  print ("\n Training...") 
  unet=load_model('model-dsbowl2018-1.h5', custom_objects={'dice_coef': dice_coef})
  checkpointer = ModelCheckpoint('model-dsbowl2018-1.h5', verbose=1, save_best_only=True)
#   results = model.fit(X_train,Y_train, validation_data=(val_data,val_label), batch_size=80, epochs=50 ,callbacks=[checkpointer])
  results = unet.fit(X_train,Y_train, validation_data=(val_data,val_label), batch_size=80, epochs=50 ,callbacks=[checkpointer])
  

if __name__ == '__main__':
  train_model()

Using TensorFlow backend.


((256, 256, 3), 'training data shape')
((256, 256, 1), 'training label shape')
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 1 448         lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, None, 1 0           conv2d_1[0][0]                   
______________________________

15678/15678 [==============================] - 358s 23ms/step - loss: 0.0703 - dice_coef: 0.8264 - val_loss: 0.0662 - val_dice_coef: 0.8336

Epoch 00001: val_loss improved from inf to 0.06615, saving model to model-dsbowl2018-1.h5
Epoch 2/50
15678/15678 [==============================] - 339s 22ms/step - loss: 0.0676 - dice_coef: 0.8329 - val_loss: 0.0648 - val_dice_coef: 0.8326

Epoch 00002: val_loss improved from 0.06615 to 0.06481, saving model to model-dsbowl2018-1.h5
Epoch 3/50
 2240/15678 [===>..........................] - ETA: 4:39 - loss: 0.0662 - dice_coef: 0.8354

15678/15678 [==============================] - 339s 22ms/step - loss: 0.0644 - dice_coef: 0.8404 - val_loss: 0.0650 - val_dice_coef: 0.8350

Epoch 00003: val_loss did not improve from 0.06481
Epoch 4/50
15678/15678 [==============================] - 339s 22ms/step - loss: 0.0623 - dice_coef: 0.8453 - val_loss: 0.0635 - val_dice_coef: 0.8376

Epoch 00004: val_loss improved from 0.06481 to 0.06349, saving model to model-dsbowl2018-1.h5
Epoch 5/50
 3840/15678 [======>.......................] - ETA: 4:06 - loss: 0.0604 - dice_coef: 0.8507

15678/15678 [==============================] - 339s 22ms/step - loss: 0.0593 - dice_coef: 0.8527 - val_loss: 0.0667 - val_dice_coef: 0.8383

Epoch 00005: val_loss did not improve from 0.06349
Epoch 6/50
15678/15678 [==============================] - 339s 22ms/step - loss: 0.0579 - dice_coef: 0.8562 - val_loss: 0.0649 - val_dice_coef: 0.8413

Epoch 00006: val_loss did not improve from 0.06349
Epoch 7/50
 4480/15678 [=======>......................] - ETA: 3:53 - loss: 0.0554 - dice_coef: 0.8627

...

15678/15678 [==============================] - 339s 22ms/step - loss: 0.0489 - dice_coef: 0.8778 - val_loss: 0.0709 - val_dice_coef: 0.8571

Epoch 00011: val_loss did not improve from 0.06349
Epoch 12/50
15678/15678 [==============================] - 339s 22ms/step - loss: 0.0475 - dice_coef: 0.8813 - val_loss: 0.0675 - val_dice_coef: 0.8545

Epoch 00012: val_loss did not improve from 0.06349
Epoch 13/50
 4560/15678 [=======>......................] - ETA: 3:51 - loss: 0.0465 - dice_coef: 0.8834

15678/15678 [==============================] - 339s 22ms/step - loss: 0.0469 - dice_coef: 0.8827 - val_loss: 0.0696 - val_dice_coef: 0.8603

Epoch 00013: val_loss did not improve from 0.06349
Epoch 14/50
15678/15678 [==============================] - 339s 22ms/step - loss: 0.0474 - dice_coef: 0.8819 - val_loss: 0.0690 - val_dice_coef: 0.8584

Epoch 00014: val_loss did not improve from 0.06349
Epoch 15/50
 4560/15678 [=======>......................] - ETA: 3:51 - loss: 0.0453 - dice_coef: 0.8889

15678/15678 [==============================] - 339s 22ms/step - loss: 0.0449 - dice_coef: 0.8876 - val_loss: 0.0700 - val_dice_coef: 0.8590

Epoch 00015: val_loss did not improve from 0.06349
Epoch 16/50
15678/15678 [==============================] - 338s 22ms/step - loss: 0.0446 - dice_coef: 0.8886 - val_loss: 0.0704 - val_dice_coef: 0.8599

Epoch 00016: val_loss did not improve from 0.06349
Epoch 17/50
 4560/15678 [=======>......................] - ETA: 3:51 - loss: 0.0440 - dice_coef: 0.8899

15678/15678 [==============================] - 339s 22ms/step - loss: 0.0439 - dice_coef: 0.8902 - val_loss: 0.0694 - val_dice_coef: 0.8581

Epoch 00017: val_loss did not improve from 0.06349
Epoch 18/50
15678/15678 [==============================] - 339s 22ms/step - loss: 0.0432 - dice_coef: 0.8921 - val_loss: 0.0710 - val_dice_coef: 0.8642

Epoch 00018: val_loss did not improve from 0.06349
Epoch 19/50
 4560/15678 [=======>......................] - ETA: 3:51 - loss: 0.0423 - dice_coef: 0.8934

15678/15678 [==============================] - 338s 22ms/step - loss: 0.0427 - dice_coef: 0.8931 - val_loss: 0.0737 - val_dice_coef: 0.8635

Epoch 00019: val_loss did not improve from 0.06349
Epoch 20/50
 8480/15678 [===============>..............] - ETA: 2:29 - loss: 0.0423 - dice_coef: 0.8947

In [0]:
file = drive.CreateFile({'parents':[{u'id': '1tIbNp1wx47x9fn9ixLLg7_lbIaSOhmOg'}]})
file.SetContentFile('model-dsbowl2018-1.h5')
file.Upload()

# u_net = load_model('model-dsbowl2018-1.h5', custom_objects={'dice_coef': dice_coef})